# Sample applications and use of the wf_graph_tools module

In [1]:
import sys
from pathlib import Path

sys.path.insert(0, "/Users/neugebauer/git_libs/pyiron_nodes")
# sys.path

In [2]:
from pyiron_nodes.development import wf_graph_tools as gt

2024-11-25 10:57:56,635 - pyiron_log - WARNING - pyiron found a 'dynamic' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().
2024-11-25 10:57:56,635 - pyiron_log - WARNING - pyiron found a 'templates' folder in the /Users/neugebauer/pyiron/resources resource directory. These are no longer supported in pyiron_base >=0.7.0. They are replaced by Project.create_job_class() and Project.wrap_python_function().


In [3]:
from pyiron_workflow import Workflow

import pyiron_nodes as pn

wf = Workflow("compute_elastic_constants")
wf.engine = pn.atomistic.engine.ase.M3GNet()
wf.bulk = pn.atomistic.structure.build.Bulk("Pb", cubic=True)
wf.input_elastic = pn.atomistic.property.elastic.InputElasticTensor()
wf.elastic = pn.atomistic.property.elastic.ElasticConstants(
    structure=wf.bulk, engine=wf.engine, parameters=wf.input_elastic
)

In [4]:
code = gt.get_code_from_wf(wf)
print(code)

from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("compute_elastic_constants")

wf.engine = pyiron_nodes.atomistic.engine.ase.M3GNet()
wf.input_elastic = pyiron_nodes.atomistic.property.elastic.InputElasticTensor()
wf.bulk = pyiron_nodes.atomistic.structure.build.Bulk(name="Pb", cubic=True)
wf.elastic = pyiron_nodes.atomistic.property.elastic.ElasticConstants(
    engine=wf.engine, structure=wf.bulk, parameters=wf.input_elastic
)



#### Test node sorting in graph

In [5]:
import numpy as np

graph = gt.get_graph_from_wf(wf)
np.random.shuffle(graph.nodes)
sorted_graph = gt.topological_sort(graph)
print(graph.nodes)

code = gt.get_code_from_graph(sorted_graph)
print(code)

[('var_bulk__cubic', True), ('bulk', 'pyiron_nodes.atomistic.structure.build.Bulk'), ('var_bulk__name', 'Pb'), ('input_elastic', 'pyiron_nodes.atomistic.property.elastic.InputElasticTensor'), ('engine', 'pyiron_nodes.atomistic.engine.ase.M3GNet'), ('elastic', 'pyiron_nodes.atomistic.property.elastic.ElasticConstants')]
from pyiron_workflow import Workflow
import pyiron_nodes

wf = Workflow("compute_elastic_constants")

wf.input_elastic = pyiron_nodes.atomistic.property.elastic.InputElasticTensor()
wf.engine = pyiron_nodes.atomistic.engine.ase.M3GNet()
wf.bulk = pyiron_nodes.atomistic.structure.build.Bulk(name="Pb", cubic=True)
wf.elastic = pyiron_nodes.atomistic.property.elastic.ElasticConstants(
    engine=wf.engine, structure=wf.bulk, parameters=wf.input_elastic
)



#### Show docstring of wf_graph_tool

In [6]:
gt?

Type:        module
String form: <module 'pyiron_nodes.development.wf_graph_tools' from '/Users/neugebauer/git_libs/pyiron_nodes/pyiron_nodes/development/wf_graph_tools.py'>
File:        ~/git_libs/pyiron_nodes/pyiron_nodes/development/wf_graph_tools.py
Docstring:  
# wf_graph_tools Module
This module provides a collection of tools to convert and manipulate workflow graphs. 
A workflow graph is represented by a set of nodes and edges, and this module offers 
various functions to transform these graphs into different representations, such as 
executable workflow code.

Additionally, this module includes helper tools to assist with tasks such as sorting 
the order of nodes in the graph, making it easier to work with and analyze workflow graphs.
The tools provided in this module are designed to be flexible and reusable, allowing users 
to easily integrate them into their own workflow processing pipelines.

# Key Features
Conversion of workflow graphs into various representations, includin

In [7]:
from pyironflow.code_editor import CodeEditorView
from IPython.display import display

# uncomment the following two lines if black window of code editor
# is not shown. Restart notebook!
# import panel
# panel.extension()


ce = CodeEditorView(code=code, width=800)

ce.code_editor
ce.show()

### Unified graph representation (discussion with Jan)

In [ ]:
graph = gt.get_graph_from_wf(wf)
graph.edges


[('engine/engine', 'elastic/engine'),
 ('bulk/structure', 'elastic/structure'),
 ('input_elastic/dataclass', 'elastic/parameters'),
 ('var_bulk__name/__str_Pb', 'bulk/name'),
 ('var_bulk__cubic/True', 'bulk/cubic')]

In [ ]:
nodes = [
    dict(label="my_label1", value="my_path", type="import_library"),
    dict(label="my_label2", value="True", type="bool"),
    dict(label="my_label3", value="[0,2,3]", type="np.array")
]

In [ ]:
import json

json.dumps(nodes)


'[{"label": "my_label1/import_library", "value": "my_path"}, {"label": "my_label2/bool", "value": "True"}, {"label": "my_label3/np.array", "value": "[0,2,3]"}]'

## Next Steps

### Include Macros/subgraphs

In [ ]:
bulk = pn.atomistic.structure.build.CubicBulkCell()

In [ ]:
bulk.graph_as_dict["nodes"]

{'/CubicBulkCell/bulk': <pyiron_nodes.atomistic.structure.build.Bulk at 0x103e20410>,
 '/CubicBulkCell/cell': <pyiron_nodes.atomistic.structure.transform.Repeat at 0x14c11ea20>,
 '/CubicBulkCell/structure': <pyiron_nodes.atomistic.structure.transform.CreateVacancy at 0x14c11eb70>}

In [ ]:
bulk.graph_as_dict["edges"]["data"]

{('/CubicBulkCell/bulk.structure',
  '/CubicBulkCell/cell.structure'): (<pyiron_workflow.mixin.injection.OutputDataWithInjection at 0x149c83ce0>, <pyiron_workflow.channels.InputData at 0x14c11e660>),
 ('/CubicBulkCell/cell.structure',
  '/CubicBulkCell/structure.structure'): (<pyiron_workflow.mixin.injection.OutputDataWithInjection at 0x14c11eb40>, <pyiron_workflow.channels.InputData at 0x14c11eae0>)}